In [ ]:
import numpy as np
import scipy as sp
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from google.colab import files
import plotly.express as px
import glob
import pandas as pd

custom_colors = ['#0e1a50', '#067a72', '#c8eae8', '#eb0046', '#ffae80', '#0050ff', '#bad8fc']
# Create a custom color palette
palette = sns.color_palette(custom_colors)
custom_palette=palette

# Get the path to the folder containing the CSV files
folder_path = f"drive/My Drive/Assignment"

# Find all CSV files in the folder
csv_files = glob.glob(folder_path + "/*.csv")
csv_files

# Load the data into three DataFrames
emails = pd.read_csv(csv_files[0], delimiter=';')
# Print the DataFrames
print("DataFrame 1:Done",emails.shape)

studenten = pd.read_csv(csv_files[1], delimiter=';')
print("\nDataFrame 2:Done",studenten.shape)

leistungen = pd.read_csv(csv_files[2], delimiter=';')
print("\nDataFrame 3:Done",leistungen.shape)

# Filter leistungen dataframe

# Convert 'emaildatum' to datetime
emails['emaildatum'] = pd.to_datetime(emails['emaildatum'], format='%d.%m.%Y')
# Filter for dates from year 2000 or later
emails = emails[emails['emaildatum'].dt.year >= 2000]

emails.shape,studenten.shape,leistungen.shape

DataFrame 1:Done (22382, 3)

DataFrame 2:Done (7376, 8)

DataFrame 3:Done (35494, 4)


((22380, 3), (7376, 8), (35494, 4))

In [ ]:

# Converting 'Datum' to datetime format
ects_df=leistungen.copy()
ects_df['Datum'] = pd.to_datetime(ects_df['Datum'], format='%d.%m.%Y').dt.date

# Grouping by 'StudiumID' and 'StudentID'
grouped_data_ects = ects_df.groupby(['StudiumID', 'StudentID']).agg({
    'Datum': lambda x: sorted(list(x)),
     'ects_gathered': lambda x: (sorted(list(x)), sum(x))
}).reset_index()


# Extracting the array of email counts and total counts into separate columns
grouped_data_ects['ects_gathered_counts'] = grouped_data_ects['ects_gathered'].apply(lambda x: x[0])
grouped_data_ects['total_ects'] = grouped_data_ects['ects_gathered'].apply(lambda x: x[1])

# Selecting and displaying the relevant columns
result_ects = grouped_data_ects[['StudiumID', 'StudentID', 'Datum', 'ects_gathered_counts', 'total_ects']]
result_ects.columns=['StudiumID', 'StudentID', 'Datum', 'ects_gathered_counts', 'total_ects']
result_ects


# Convert 'emaildatum' to datetime format
emails['emaildatum'] = pd.to_datetime(emails['emaildatum'], format='%Y-%m-%d').dt.date

# Grouping by 'studentid' and aggregating email dates in increasing order
grouped_data_email = emails.groupby('studentid').agg({
    'emaildatum': lambda x: sorted(list(x)),
    'anzahl_emails': lambda x: (sorted(list(x)), sum(x))  # Sorting and summing the number of emails
}).reset_index()

# Extracting the array of email counts and total counts into separate columns
grouped_data_email['email_counts'] = grouped_data_email['anzahl_emails'].apply(lambda x: x[0])
grouped_data_email['total_emails'] = grouped_data_email['anzahl_emails'].apply(lambda x: x[1])

# Selecting and displaying the relevant columns
result_email = grouped_data_email[['studentid', 'emaildatum', 'email_counts', 'total_emails']]
result_email.columns=[ 'StudentID','emaildatum', 'email_counts', 'total_emails']
result_email

# Merging the two datasets
merged_results = pd.merge(result_ects, result_email, on='StudentID', how='outer')
merged_results


# Function to extract the last word from the 'Studiengang' column
def extract_last_word(studiengang):
    return studiengang.split()[-1]

# Applying the function to create a new column with the extracted last word
studenten['Field'] = studenten['Studiengang'].apply(extract_last_word)
studenten['Studiumbeginn'] = pd.to_datetime(studenten['Studiumbeginn'], format='%d.%m.%Y').dt.date
#studenten['Kuendigungsdatum'] = pd.to_datetime(studenten['Kuendigungsdatum'], format='%d.%m.%Y').dt.date
# Merging the two datasets on 'StudiumID' and 'StudentID'
merged_final = pd.merge(studenten, merged_results, on=['StudiumID', 'StudentID'], how='outer')
merged_final



,StudiumID,StudentID,Studiumbeginn,Alter,Geschlecht,Studiengang,Leistungspunkte_zum_Abschluss,Kuendigungsdatum,Field,Datum,ects_gathered_counts,total_ects,emaildatum,email_counts,total_emails
0,31600509.0,31222461,2019-05-07,50.0,f,Bachelor International Management berufsbeglei...,90.0,06.05.2021,berufsbegleitend,NaN,NaN,NaN,NaN,NaN,NaN
1,31679100.0,31258011,2019-07-20,30.0,f,Fernstudium Bachelor of Arts Public Relations,180.0,20.04.2021,Relations,NaN,NaN,NaN,NaN,NaN,NaN
2,31631508.0,31237821,2019-05-25,28.0,f,Fernstudium Bachelor of Arts Pflegemanagement,180.0,07.04.2021,Pflegemanagement,"[2019-08-02, 2019-09-22, 2019-10-15, 2019-10-1...","[5, 5, 5, 5, 5, 5, 5, 5]",40.0,"[2019-07-26, 2019-10-18, 2019-10-23, 2019-11-0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",11.0
3,31685331.0,31261038,2019-08-01,26.0,m,Fernstudium Bachelor of Arts Pflegepädagogik,120.0,07.04.2021,Pflegepädagogik,NaN,NaN,NaN,NaN,NaN,NaN
4,31655100.0,31018401,2019-06-27,27.0,f,Fernstudium Bachelor of Arts Physiotherapie,180.0,07.04.2021,Physiotherapie,"[2019-08-01, 2019-10-20, 2019-10-21, 2020-06-0...","[2, 5, 5, 5, 5, 6, 20]",48.0,"[2019-08-09, 2019-08-21, 2019-09-13, 2019-09-1...","[1, 1, 1, 1, 1, 1, 1, 1, 15]",23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8922,NaN,31300797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2019-10-22, 2019-10-24, 2019-11-07, 2020-06-0...","[1, 1, 1, 1, 1]",5.0
8923,NaN,31303572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2019-11-03, 2019-11-06, 2021-03-28, 2021-04-0...","[1, 1, 1, 1, 1]",5.0
8924,NaN,31340367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2019-12-15, 2019-12-20, 2020-05-24]","[1, 1, 1]",3.0
8925,NaN,31344105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[2020-09-30],[1],1.0


In [ ]:
# prompt: download dataframe in excel file

merged_final.to_excel('merged_final_result.xlsx', index=False)
files.download('merged_final_result.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>